In [1]:
import numpy as np

In [2]:
m=5
n=10

S = np.random.rand(m,1)
A = np.random.rand(m,m)
A, _ = np.linalg.qr(A) #ortogonalna da nam norme vektora ne eksplodiraju

for i in range(n):
    S = np.hstack((S, A@(S[:,-1].reshape(-1,1))))

S_bez_suma = S
S = S+np.random.normal(0, 0.08, S.shape)
X = S[:, :-1]
Y = S[:, 1:]

In [14]:
def stln_dmd_v3(Xt, Yt, tol=1e-12, max_iter=5000, w=1000):
    (n,m) = Xt.shape
    alpha = np.zeros((n*m, 1))
    beta = np.zeros((m,1))

    At = np.linalg.pinv(Xt)@Yt
    r0 = Yt - Xt@At
    r0 = r0.reshape(-1,1, order='F')


    beta = r0[-m:, :]
    br=0
    for i in range((n-1)*m):
        if(br%n == 0):
            br+=1
        alpha[br,0] = r0[i,0]

    indeksi=[]
    for i in range(m*n):
        if(i%n != 0):
            indeksi += [i]
    
    
    E = np.zeros_like(Xt)

    P_ind = np.zeros((m*n, m*n))
    br=0
    for i in range(P_ind.shape[1]-m):
        if(br%n==0):
            br=br+1
        P_ind[i, br] = -1.0
        br=br+1

    At_norm = 1
    alpha_norm = 1
    beta_norm = 1

    M1 = np.kron(np.eye(m), Xt+E)
    #M2 = np.kron(At, np.eye(n))+P_ind
    M2 = np.kron(At.T, np.eye(n))+P_ind
    M3 = np.vstack((np.zeros(((n-1)*m, m)), np.eye(m)))

    M = np.hstack((M1, M2, -M3))

    D1 = np.eye(n*m)
    D1[indeksi,indeksi] = 2.0
    D1 = np.hstack((np.zeros((n*m,m*m)), D1, np.zeros((n*m, m))))
    D2 = np.hstack((np.zeros((m, m*(m+n))), np.eye(m)))

    M = np.vstack((w*M,D1,D2))

    alpha_poc = alpha
    beta_poc = beta
    At_poc = At.reshape(-1,1, order='F')
    print(np.linalg.norm(r0))

    rhs = np.vstack((w*r0, alpha, beta)) 

    iter=0

    while((At_norm > tol or alpha_norm > tol or beta_norm > tol) and iter<max_iter):
        sol,_,_,_ = np.linalg.lstsq(M, rhs)

        At = sol[:m*m,:]
        alpha = sol[m*m:m*(m+n),:]
        beta = sol[-m:,:]

        At_poc += At
        alpha_poc += alpha
        beta_poc += beta

        At_norm = np.linalg.norm(At)
        alpha_norm = np.linalg.norm(alpha)
        beta_norm = np.linalg.norm(beta)

        At_norm = np.linalg.norm(At)/np.linalg.norm(At_poc)
        alpha_norm = np.linalg.norm(alpha)/np.linalg.norm(alpha_poc)
        beta_norm = np.linalg.norm(beta)/np.linalg.norm(beta_poc)

        print(At_norm, alpha_norm, beta_norm)

        #r0 = r0 -np.kron(np.eye(m),(Xt+alpha_poc.reshape(n,m, order='F')))@At - np.kron(At_poc.reshape(m,m, order='F'), np.eye(n))@alpha -P_ind@alpha + M3 @ beta
        r0 = r0 -np.kron(np.eye(m),(Xt+alpha_poc.reshape(n,m, order='F')))@At - np.kron(At_poc.reshape(m,m), np.eye(n))@alpha -P_ind@alpha + M3 @ beta

        print("r0:",np.linalg.norm(r0))

        #provjeri..
        E = alpha_poc.reshape(n,m, order='F') 
        M1 = np.kron(np.eye(m), Xt+E)
        #M2 = np.kron(At_poc.reshape(m,m, order='F').T, np.eye(n))+P_ind
        M2 = np.kron(At_poc.reshape(m,m), np.eye(n))+P_ind #At_poc.reshape(m,m) isto kao (At_poc.T).reshape(m,m, order='F') 
        M = np.hstack((M1, M2, -M3))
        M= np.vstack((w*M,D1,D2))


        rhs = np.vstack((w*r0, -alpha_poc, -beta_poc)) ##ili bez poc?

        iter+=1

    Ey = np.vstack(((-P_ind[:-m,:]@alpha_poc[:,0]).reshape(n-1,m, order='F'), beta_poc.reshape(1,-1, order='F')))
    print(beta_poc.reshape(1,-1, order='F'))
    print("broj iteracija:",iter)

    return At_poc.reshape(m,m), alpha_poc.reshape(m,n), Ey.T


In [15]:
A_rek, Ex, Ey = stln_dmd_v3(X.T, Y.T, w=1000, tol=1e-8)

0.464593598821082
0.15096365198071876 0.9835576384006999 2.114277381136354
r0: 0.22707999477500682
0.22621068006211326 0.5204364553857915 1.6355948460710616
r0: 0.09306981741419947
0.09699332464333403 0.20295078436891778 0.4264640631192178
r0: 0.02176102374693391
0.07180657909526768 0.1443513153157309 0.2932431528327433
r0: 0.008979338238263947
0.06338943449620645 0.10264649630254977 0.15634199589375397
r0: 0.006017074607370743
0.051841427428728985 0.0835838275124938 0.13190015411273595
r0: 0.003918512025883186
0.04122807618047341 0.06623674168262765 0.0965676938670754
r0: 0.0024764857696521697
0.03198523602752438 0.051417291399073646 0.07109047385445148
r0: 0.001507219090600857
0.024457837477141423 0.03929284923988815 0.05190890627389901
r0: 0.0008962014065404608
0.018538268173419123 0.029724199915504096 0.03783530712371213
r0: 0.0005249622036483544
0.013979637163652324 0.02235564774275864 0.027587494411107274
r0: 0.0003044819392745223
0.01051063670838336 0.016761967963574636 0.020147

In [16]:
print(np.linalg.norm(Y@np.linalg.pinv(X)-A), np.linalg.norm(A_rek-A))

1.0447320968508034 1.9573238807280147


In [17]:
np.linalg.norm(A_rek@(X+Ex)-Y-Ey) ##izracunato reda 10^-7... krivo racunanje r?

np.float64(0.606421924193392)

In [18]:
print(X+Ex)
print(S_bez_suma[:,:-1])


[[ 0.79022548  0.193766   -0.5889714   0.16544962  0.74216056 -1.05291477
   0.23674557  0.65613535 -0.73765528 -0.06819075]
 [ 0.25105588 -0.37889363  0.30752755 -0.55937651  0.53752503 -0.49062369
   0.23685875 -0.43849715  0.4853693  -0.60144398]
 [ 0.58627712 -0.38675713 -0.43889663  0.92570256 -0.71398667 -0.49361254
   0.61962519 -0.08745006 -0.79256663  0.9480352 ]
 [ 0.20545903 -0.73603739  0.73371999 -0.11189352 -0.63835232  0.21123823
   0.42647419 -0.81612075  0.46405205  0.40707506]
 [ 0.74292603 -0.34764731  0.36810706  0.33851096 -0.00144124 -0.14500353
   0.76154279 -0.28455611 -0.01960261  0.29116913]]
[[ 0.81027744  0.15555423 -0.61232288  0.24315509  0.68783639 -0.89562236
   0.30010507  0.62590683 -0.65702512 -0.02792315]
 [ 0.27623701 -0.35234964  0.2823836  -0.59750549  0.52768039 -0.59812727
   0.31483614 -0.44289558  0.41555769 -0.69644362]
 [ 0.60571953 -0.57658999 -0.50885303  0.94670019 -0.67326728 -0.45085561
   0.69067792 -0.22600488 -0.84526005  0.83062623]

In [19]:
Y+Ey - A_rek@(X+Ex)

array([[ 5.20042409e-02,  2.11397046e-02,  9.58680586e-03,
        -1.53706964e-02, -1.00370290e-02, -4.64035968e-03,
        -9.55843482e-05,  2.77956932e-04, -8.04346133e-03,
        -4.03536257e-02],
       [ 2.90482149e-02, -7.47173715e-02,  9.07832064e-02,
        -2.83540486e-02,  7.29296230e-02,  4.28248288e-03,
        -5.68124609e-02,  7.02795428e-03,  3.56726904e-02,
        -2.97902672e-02],
       [-5.22793758e-02,  5.63997505e-02, -6.32099904e-02,
         3.11391317e-02, -3.25321279e-02, -1.66440248e-02,
         8.65308251e-02, -7.84335381e-02,  3.96096659e-02,
        -1.07791766e-01],
       [ 1.62093682e-01,  1.05545710e-01,  7.81269419e-02,
        -1.20478321e-01, -2.74511881e-01, -1.16945995e-02,
         1.22670046e-02,  1.20934991e-01,  4.91348498e-02,
         1.80892085e-01],
       [ 2.48433502e-02,  3.70009416e-02, -3.29479992e-02,
        -6.23906022e-02,  2.49549289e-02, -1.67850884e-01,
        -8.46110912e-02, -1.04003683e-01,  8.07258874e-03,
         2.

In [20]:
Ex[:, 1:]


array([[ 0.10548545,  0.07421292, -0.01619905, -0.0374419 , -0.03685779,
         0.00167623, -0.04622143, -0.04149527,  0.01257309],
       [ 0.03072034,  0.01657412,  0.09029588,  0.00503728,  0.0296757 ,
        -0.04945865, -0.05895525,  0.00355535,  0.0052712 ],
       [-0.03333988,  0.03128138, -0.01288002, -0.00972226,  0.02384947,
        -0.02633462,  0.0733688 ,  0.0034951 , -0.00437096],
       [ 0.07397544, -0.03564163,  0.0803357 , -0.07684168, -0.13813357,
         0.00978134,  0.04530038,  0.12653194,  0.0025684 ],
       [ 0.03493811,  0.02145183, -0.04263082, -0.09136549, -0.0023665 ,
        -0.00606215, -0.02014026, -0.0287957 , -0.01870114]])

In [21]:
Ey

array([[ 0.10548545,  0.07421292, -0.01619905, -0.0374419 , -0.03685779,
         0.00167623, -0.04622143, -0.04149527,  0.01257309, -0.01659709],
       [ 0.03072034,  0.01657412,  0.09029588,  0.00503728,  0.0296757 ,
        -0.04945865, -0.05895525,  0.00355535,  0.0052712 , -0.00277775],
       [-0.03333988,  0.03128138, -0.01288002, -0.00972226,  0.02384947,
        -0.02633462,  0.0733688 ,  0.0034951 , -0.00437096, -0.00332321],
       [ 0.07397544, -0.03564163,  0.0803357 , -0.07684168, -0.13813357,
         0.00978134,  0.04530038,  0.12653194,  0.0025684 ,  0.07579315],
       [ 0.03493811,  0.02145183, -0.04263082, -0.09136549, -0.0023665 ,
        -0.00606215, -0.02014026, -0.0287957 , -0.01870114,  0.10889534]])

In [22]:
print(A)
print(A_rek)
print(Y@np.linalg.pinv(X))

[[-0.5721609  -0.17779008  0.50606211  0.04805133  0.61855865]
 [-0.10166564 -0.59673533  0.46461203 -0.1072735  -0.63733751]
 [-0.59760676  0.27788619 -0.16260366  0.62324474 -0.38829301]
 [-0.55100684  0.0455563  -0.39269817 -0.72522503 -0.11896595]
 [-0.0394161  -0.73006953 -0.58939376  0.26795171  0.2150854 ]]
[[-1.00933658  0.36571039  1.01367272 -0.36659978  0.4422777 ]
 [-0.11340063 -0.60418221  0.41941187 -0.22576152 -0.49285273]
 [ 0.27177561 -0.36251352 -0.71274042  1.53128178 -0.47781727]
 [-0.46471727  0.20305028 -0.33138266 -0.75890395 -0.31183572]
 [-0.36363032 -0.77305044 -0.5934055  -0.1452303   0.65508148]]
[[-0.86646583  0.2195361   0.88575138 -0.18669276  0.39901913]
 [ 0.0232009  -0.6778715   0.29698866 -0.05236148 -0.55937408]
 [-0.36530694  0.19481178 -0.27517235  0.75445949 -0.27585996]
 [-0.27836107  0.00990816 -0.43817112 -0.50701009 -0.38179666]
 [-0.29035018 -0.67717348 -0.49669173 -0.04226772  0.48301619]]
